#### Cost of Normalization


##### Senario 1: $(D^{\frac{-1}{2}}AD^{\frac{-1}{2}}X)W$
Do  $D^{\frac{-1}{2}}AD^{\frac{-1}{2}}$ first, then do $(AX)W$, the number of operation would be:
- $D^{\frac{-1}{2}}AD^{\frac{-1}{2}}$: $2nnz$ Mult
- [SpmDm] AX: $nnz*F_{in}$ Mult and $nnz*F_{in}$ Add
- [DmDm] (AX)W: $N*F_{in}*F_{out}$ Mult and $N*F_{in}*F_{out}$ Add

In total it's $2nnz + nnz*F_{in} + N*F_{in}*F_{out}$ Mult and $nnz*F_{in} + N*F_{in}*F_{out}$ Add

##### Senario 2: $(D^{\frac{-1}{2}}AD^{\frac{-1}{2}})(XW)$
Do  $D^{\frac{-1}{2}}AD^{\frac{-1}{2}}$ first, then do $A(XW)$, the number of operation would be:
- $D^{\frac{-1}{2}}AD^{\frac{-1}{2}}$: $2nnz$ Mult
- [DmDm] XW: $N*F_{in}*F_{out}$ Mult and $N*F_{in}*F_{out}$ Add
- [SpmDm] A(XW): $nnz*F_{out}$ Mult and $nnz*F_{out}$ Add

In total it's $2nnz + nnz*F_{out} + N*F_{in}*F_{out}$ Mult and $nnz*F_{out} + N*F_{in}*F_{out}$ Add

##### Senario 3: $D^{\frac{-1}{2}}(AD^{\frac{-1}{2}}XW)$
Do $AD^\frac{-1}{2}$ only, and multiply $D^\frac{-1}{2}$ after aggeregation:
- $AD^\frac{-1}{2}$: $nnz$ Mult
- Aggr: $nnz*F_{in}$ Add
- Comb: $N*F_{in}*F_{out}$ Mult and $N*F_{in}*F_{out}$ Add
- Multiply $D^\frac{-1}{2}$: $N*F_{out}$ Mult

In total it's $nnz+N*F_{in}*F_{out}+N*F_{out}$ Mult and $nnz*F_{in} + N*F_{in}*F_{out}$ Add

##### Senario 4: $D^{\frac{-1}{2}}((AD^{\frac{-1}{2}})(XW))$
Do $AD^\frac{-1}{2}$ only, and multiply $D^\frac{-1}{2}$ after aggeregation:
- $AD^\frac{-1}{2}$: $nnz$ Mult
- Aggr: $nnz*F_{in}$ Add
- Comb: $N*F_{in}*F_{out}$ Mult and $N*F_{in}*F_{out}$ Add
- Multiply $D^\frac{-1}{2}$: $N*F_{out}$ Mult

In total it's $nnz+N*F_{in}*F_{out}+N*F_{out}$ Mult and $nnz*F_{in} + N*F_{in}*F_{out}$ Add

###### Example:
dataset Cora: $N=2708$, $nnz=10556$, layer 1: $F_{in}=1433$, $F_{out}=16$

S1: 77.25M Mult, 77.23M Add

S2: 62.28M Mult, 62.26M Add

S3: 62.14M Mult (80% of S1), 77.23M Add

dataset Reddit: $N=233K$, $nnz=114.6M$, layer 1: $F_{in}=602$, $F_{out}=16$

S1: 71.24B Mult, 71.24B Add

S2:

S3: 2.24B Mult (3.14% of S1), 71.24B Add


In [1]:
import numpy as np
from numpy.linalg import matrix_power
import math

A = np.random.choice(2, (5,5), p=[0.6, 0.4])
D = np.zeros((5,5), dtype=int)
DNorm = np.zeros((5,5), dtype=float)
for i in range(D.shape[0]):
    D[i][i] = sum(A[i])
print("A = \n", A)
print("D = \n", D)
print("")

# Normalize
for i in range(D.shape[0]):
    if D[i][i]:
        DNorm[i][i] = 1/math.sqrt(D[i][i])
ANorm1 = np.matmul(DNorm, A)
ANorm2 = np.matmul(ANorm1, DNorm)
print("DNorm = \n", DNorm)
print("ANorm1 = \n", ANorm1)
print("ANorm2 = \n", ANorm2)

A = 
 [[1 0 0 1 0]
 [1 0 0 0 1]
 [1 0 1 0 1]
 [1 0 1 1 1]
 [1 0 0 1 1]]
D = 
 [[2 0 0 0 0]
 [0 2 0 0 0]
 [0 0 3 0 0]
 [0 0 0 4 0]
 [0 0 0 0 3]]

DNorm = 
 [[0.70710678 0.         0.         0.         0.        ]
 [0.         0.70710678 0.         0.         0.        ]
 [0.         0.         0.57735027 0.         0.        ]
 [0.         0.         0.         0.5        0.        ]
 [0.         0.         0.         0.         0.57735027]]
ANorm1 = 
 [[0.70710678 0.         0.         0.70710678 0.        ]
 [0.70710678 0.         0.         0.         0.70710678]
 [0.57735027 0.         0.57735027 0.         0.57735027]
 [0.5        0.         0.5        0.5        0.5       ]
 [0.57735027 0.         0.         0.57735027 0.57735027]]
ANorm2 = 
 [[0.5        0.         0.         0.35355339 0.        ]
 [0.5        0.         0.         0.         0.40824829]
 [0.40824829 0.         0.33333333 0.         0.33333333]
 [0.35355339 0.         0.28867513 0.25       0.28867513]
 [0.40824

#### Sparsity of different dataset

In [2]:
import torch
import numpy as np
from torch_geometric.datasets import TUDataset # IMDB-BINARY, REDDIT-BINARY or PROTEINS
from torch_geometric.datasets import Reddit # IMDB-BINARY, REDDIT-BINARY or PROTEINS
from torch_geometric.datasets import Planetoid # Cora, CiteSeer and PubMed
from torch_geometric.datasets import CitationFull # CoraFull, DBLP
from torch_geometric.datasets import GNNBenchmarkDataset # PATTERN, CLUSTER, MNIST, CIFAR10, TSP, CSL
from torch_geometric.datasets import Yelp # Yelp
from torch_geometric.datasets import Planetoid # Cora, CiteSeer and PubMed

from torch_geometric.data import Data
from torch_geometric.data import DataLoader

cora      = Planetoid(root='/tmp/Cora', name='Cora')                   # Citation networks
citeseer  = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')           # Citation networks
pubmed    = Planetoid(root='/tmp/PubMed', name='PubMed')               # Citation networks
corafull  = CitationFull(root='/tmp/CoraFull', name='cora')            # Citation networks
dblp      = CitationFull(root='/tmp/DBLP', name='DBLP')                # Citation networks

collab    = TUDataset(root='/tmp/COLLAB', name='COLLAB')               # Social networks
imdb      = TUDataset(root='/tmp/IMDB-BINARY', name='IMDB-BINARY')     # Social networks
# reddit    = TUDataset(root='/tmp/REDDIT-BINARY', name='REDDIT-BINARY') # Social networks
reddit    = Reddit(root='/tmp/Reddit') # Social networks
proteins  = TUDataset(root='/tmp/PROTEINS', name='PROTEINS')           # Bioinformatics

pattern   = GNNBenchmarkDataset(root='/tmp/PATTERN', name='PATTERN')   # 
cluster   = GNNBenchmarkDataset(root='/tmp/CLUSTER', name='CLUSTER')   # 
mnist     = GNNBenchmarkDataset(root='/tmp/MNIST', name='MNIST')       # 
cifar10   = GNNBenchmarkDataset(root='/tmp/CIFAR10', name='CIFAR10')   # 
tsp       = GNNBenchmarkDataset(root='/tmp/TSP', name='TSP')           # 
csl       = GNNBenchmarkDataset(root='/tmp/CSL', name='CSL')           # 

yelp      = Yelp(root='/tmp/Yelp')                                     # 




def get_density(data):
    return np.count_nonzero(data.cpu().detach().numpy())/data.shape[0]/data.shape[1]

print(f"Cora Feature ({cora.data.x.shape}) Density: ", get_density(cora.data.x))
print(f"CiteSeer Feature ({citeseer.data.x.shape}) Density: ", get_density(citeseer.data.x))
print(f"PubMed Feature ({pubmed.data.x.shape}) Density: ", get_density(pubmed.data.x))
print(f"CoraFull Feature ({corafull.data.x.shape}) Density: ", get_density(corafull.data.x))
print(f"DBLP Feature ({dblp.data.x.shape}) Density: ", get_density(dblp.data.x))
print(f"Yelp Feature ({yelp.data.x.shape}) Density: ", get_density(yelp.data.x))

# print(f"Proteins Feature ({proteins.data.x.shape}) Density: ", get_density(proteins.data.x))
# print(f"Pattern Feature ({pattern.data.x.shape}) Density: ", get_density(pattern.data.x))
# print(f"Pattern Feature ({cluster.data.x.shape}) Density: ", get_density(cluster.data.x))
# print(f"Pattern Feature ({mnist.data.x.shape}) Density: ", get_density(mnist.data.x))
# print(f"Pattern Feature ({cifar10.data.x.shape}) Density: ", get_density(cifar10.data.x))
# print(f"Pattern Feature ({tsp.data.x.shape}) Density: ", get_density(tsp.data.x))
# print(f"Pattern Feature ({csl.data.x.shape}) Density: ", get_density(csl.data.x))

# print(yelp.__dict__)
# print(dir(imdb))
# print(reddit.num_node_features)
# print(reddit.num_edge_features)
# print(reddit.num_features)

# print("IMDB Feature Density: ", get_density(imdb.data.x))
# print("COLLAB Feature Density: ", get_density(collab.data.x))
# print(reddit.__dict__)
# print("Reddit Feature Density: ", get_density(reddit.data.x))
# print("Proteins Feature Density: ", get_density(proteins.data.x))

Cora Feature (torch.Size([2708, 1433])) Density:  0.012682692515830173
CiteSeer Feature (torch.Size([3327, 3703])) Density:  0.008536202581826887
PubMed Feature (torch.Size([19717, 500])) Density:  0.10022123041030583
CoraFull Feature (torch.Size([19793, 8710])) Density:  0.006529172805355173
DBLP Feature (torch.Size([17716, 1639])) Density:  0.0031750356895336373
Yelp Feature (torch.Size([716847, 300])) Density:  0.999902350152822


AttributeError: 'NoneType' object has no attribute 'cpu'

In [6]:
# Time each layer
import torch
import torch.nn.functional as F
import time
import numpy as np
from pprint import pprint

from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

from torchsummary import summary

dataset = None
data = None

conv1_time = 0
relu_time = 0
conv2_time = 0

class Net1(torch.nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True) # 1433->16
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True)  # 16->7

    def forward(self, data):
        global conv1_time, relu_time, conv2_time
        x, edge_index = data.x, data.edge_index
        
        start = time.time()
        x = self.conv1(x, edge_index)
        end = time.time()
        conv1_time += (end-start)*1000
        
        start = time.time()
        x = F.relu(x)
        end = time.time()
        relu_time += (end-start)*1000
        
        start = time.time()
        x = self.conv2(x, edge_index)
        end = time.time()
        conv2_time += (end-start)*1000
        return F.log_softmax(x, dim=1)

# Training
model = None
optimizer = None

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    return float(loss)
    
for i in [cora, corafull, citeseer, pubmed, dblp]:
    global dataset, data, model, optimizer
    dataset = i
    data = dataset[0]
    model = Net1()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
#     pprint(dataset.__dict__)
#     pprint(dataset)
    print(dataset.name)
#     print(data)
    conv1_time = 0
    relu_time = 0
    conv2_time = 0
    for epoch in range(20):
        loss = train(data)
    print(f"Conv1 time: {conv1_time} ms")
    print(f"Relu time: {relu_time} ms")
    print(f"Conv2 time: {conv2_time} ms")
    model.eval()
    _, pred = model(data).max(dim=1)
    if hasattr(data, "test_mask"):
        correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        acc = correct / int(data.test_mask.sum())
    else:
        correct = int(pred.eq(data.y).sum().item())
        acc = correct / int(data.y.shape[0])
    print('Accuracy: {:.4f}'.format(acc))
    print("")


Cora
Conv1 time: 68.13454627990723 ms
Relu time: 1.1546611785888672 ms
Conv2 time: 15.990257263183594 ms
Accuracy: 0.9090

cora
Conv1 time: 2049.2217540740967 ms
Relu time: 2.872467041015625 ms
Conv2 time: 1973.1969833374023 ms
Accuracy: 0.5977

CiteSeer
Conv1 time: 84.1214656829834 ms
Relu time: 1.5490055084228516 ms
Conv2 time: 14.474868774414062 ms
Accuracy: 0.8740

PubMed
Conv1 time: 288.62690925598145 ms
Relu time: 3.010988235473633 ms
Conv2 time: 48.21443557739258 ms
Accuracy: 0.8050

dblp
Conv1 time: 505.56349754333496 ms
Relu time: 2.6674270629882812 ms
Conv2 time: 59.56864356994629 ms
Accuracy: 0.8405



In [ ]:
%%capture output
import torch.nn.utils.prune as prune
import pickle 
import numpy as np

dataset = None
data = None

conv1_time = 0
relu_time = 0
conv2_time = 0

class Net2(torch.nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, cached=True) # 1433->16
        self.conv2 = GCNConv(16, dataset.num_classes, cached=True)  # 16->7

    def forward(self, data):
        global conv1_time, relu_time, conv2_time
        x, edge_index = data.x, data.edge_index
        
        start = time.time()
        x = self.conv1(x, edge_index)
        end = time.time()
        conv1_time += (end-start)*1000
        
        start = time.time()
        x = F.relu(x)
        end = time.time()
        relu_time += (end-start)*1000
        
        start = time.time()
        x = self.conv2(x, edge_index)
        end = time.time()
        conv2_time += (end-start)*1000
        return F.log_softmax(x, dim=1)

# Training
model = None
optimizer = None

def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    return float(loss)

def eval_(data): # eval is reseved word
    model.eval()
    print(data.edge_index.shape)
    _, pred = model(data).max(dim=1)
    if hasattr(data, "test_mask"):
        correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        acc = correct / int(data.test_mask.sum())
    else:
        correct = int(pred.eq(data.y).sum().item())
        acc = correct / int(data.y.shape[0])
    return acc

def sample(edge_index, num): # edge sampling
    edge_index = edge_index.detach().cpu().numpy()
    with open('reddit_edge_index_split.pkl', 'rb') as f:
        index_split = pickle.load(f)
    index_split.insert(0, 0)
    index_split.append(len(edge_index[0]))
#     for i in range(1, len(edge_index[0])):
#         if edge_index[0][i] != edge_index[0][i-1]:
#             split_index.append(i)
    ret = [[],[]]
    for i in range(len(index_split)-1):
        selection = list(np.random.choice(edge_index[1][index_split[i]:index_split[i+1]], num))
        ret[0]+=[i]*num
        ret[1]+=selection
#     print(len(ret[0]))
#     print(len(ret[1]))
    ret = torch.LongTensor(ret)
    return ret
    
# for i in [cora, corafull, citeseer, pubmed, dblp]:
for i in [reddit]:
    for _ in range(10):
        global dataset, data, model, optimizer
        dataset = i
        data = dataset[0]
        full_edge_index = data.edge_index
        sample_edge_index = sample(full_edge_index, 25)
#         data.edge_index = sample(full_edge_index, 25)
        model = Net2()
#         prune.l1_unstructured(model.conv1, name='weight', amount=0.9)
#         prune.l1_unstructured(model.conv2, name='weight', amount=0.8)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

#         pprint(dataset.__dict__)
#         pprint(data)
#         print(dataset.name)
#         print(data.edge_index)
#         print(data.edge_index.shape)
        conv1_time = 0
        relu_time = 0
        conv2_time = 0
        for epoch in range(5000):
            print("Epoch ", epoch)
#             data.edge_index = sample(full_edge_index, 25) # resample every epoch
            loss = train(data)
            if not epoch%10: # evaluate every 10 epoch
                acc = eval_(data)
                print("Accuracy w/o sample: ", acc)
                
                data.edge_index = sample_edge_index
                acc = eval_(data)
                print("Accuracy w/ sample: ", acc)
                data.edge_index = full_edge_index
#                 if acc > 0.90:
#                     print(f"epoch {epoch}")
#                     break
        print(f"Conv1 time: {conv1_time} ms")
        print(f"Relu time: {relu_time} ms")
        print(f"Conv2 time: {conv2_time} ms")


        print('Accuracy: {:.4f}'.format(acc))
        print("")



Epoch  0
torch.Size([2, 114615892])
Accuracy w/o sample:  0.09511157388291475
torch.Size([2, 5824125])
Accuracy w/ sample:  0.09511157388291475
Epoch  1
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
torch.Size([2, 114615892])
Accuracy w/o sample:  0.4510529055885679
torch.Size([2, 5824125])
Accuracy w/ sample:  0.4510529055885679
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Epoch  15
Epoch  16
Epoch  17
Epoch  18
Epoch  19
Epoch  20
torch.Size([2, 114615892])
Accuracy w/o sample:  0.7291707807478951
torch.Size([2, 5824125])
Accuracy w/ sample:  0.7291707807478951
Epoch  21
Epoch  22
Epoch  23
Epoch  24
Epoch  25
Epoch  26
Epoch  27
Epoch  28
Epoch  29
Epoch  30
torch.Size([2, 114615892])
Accuracy w/o sample:  0.8397393318133674
torch.Size([2, 5824125])
Accuracy w/ sample:  0.8397393318133674
Epoch  31
Epoch  32
Epoch  33
Epoch  34
Epoch  35
Epoch  36
Epoch  37
Epoch  38
Epoch  39
Epoch  40
torch.Size([2, 114615892])
Accuracy w/o sample:  0.886397501032260